# Opcion descargar data de cierta faena

Si scaa actualiza una muestra, tiene que actualizarse con la fecha de la fecha muestra, por lo que tiene que haber un gap

In [76]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import sys
from tqdm import tqdm
import requests
import json
import importlib
import os
import datetime as dt
import xlrd

ruta_archivo = os.getcwd()
ruta_base =os.path.dirname(ruta_archivo)

sys.path.append(ruta_base)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

with open(ruta_base +'//config.json', "r") as configFile:
    config = json.load(configFile)

# faenas
faenas = config['faenas']

# Actualizaciones
updates = config['updates']

## Parametros

In [77]:
# Se guardara en la carpeta '02_Archivos' 
carpeta_scaa = 'DatosScaa'
carpeta_limites = 'DatosLimites'

hoy = dt.date.today()

fecha_actualizar = updates['last_date_scaa']
fecha_actualizar = dt.datetime.strptime(fecha_actualizar, '%d/%m/%Y').date()
# hasta = '01/03/2023'
intervalo = 20 # dias
# intervalo en dias

In [78]:
hasta = hoy.strftime('%d/%m/%Y')



fecha_intervalo =  (hoy- dt.timedelta(intervalo))

# Excel opcional con los datos mas recientes
desde_actual = '2020-01-01'

In [79]:
desde = min(fecha_actualizar,fecha_intervalo).strftime('%d/%m/%Y')

In [80]:
# Fechas de descarga de datos
# Datos desde 2013
# desde = '01/01/2013'
# hasta = '01/01/2014'

desde = '01/01/2020'
hasta = '01/01/2021'


## Utilities

In [81]:
def ruta_descarga(faena,desde,hasta):
    # Descarga de pagina Scaa (datos mas recientes)
    # https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=11&desde=01/01/2023&hasta=10/01/2023&flota=-1&alerta=1&componente=M09
    return "https://scaaweb.komatsu.cl/generahistorico_faena.php?faena="+ faena + "&desde=" + desde +"&hasta=" +hasta + "&flota=-1&alerta=1&componente=M09"

def si_carpeta(carpeta):
    guardar_carpeta = os.path.abspath(carpeta)
    # directory = save_path + '\\' + carpeta_guardado 
    if not os.path.exists(guardar_carpeta):
        os.makedirs(guardar_carpeta)

In [82]:
# direccion de archivos
archivos_scaa = ruta_base + '/../02_Archivos/' + carpeta_scaa
archivos_temp = archivos_scaa + '/temps'
archivos_limites = ruta_base + '/../02_Archivos/' + carpeta_limites

si_carpeta(archivos_scaa)
si_carpeta(archivos_temp)
si_carpeta(archivos_limites)

In [83]:
block_size = 1024 #1 Kibibyte

temp_name = 'temp'

# cabezal del metodo get
headers = {'User-Agent': 'Mozilla/5.0',
            'Connection': 'keep-alive',
            'Host': 'scaaweb.komatsu.cl'}

In [84]:
# request para obtener las cookies
r1 = requests.post('https://scaaweb.komatsu.cl/insert.php')
timeout = 60*5

for i,(faena, id_faena) in enumerate(faenas.items()):
    # if faena in ['Chuquicamata']: #
    # ['Los Pelambres', 'Andina', 'Radomiro Tomic', 'Cerro Negro Norte','Sierra Gorda', 'Gaby', 'Quebrada Blanca', 'Collahuasi', 'Caserones',
    # 'Ministro Hales', 'Chuquicamata', 'Antucoya','SPENCE', 'Lomas Bayas', 'Centinela']
        url = ruta_descarga(str(id_faena),desde, hasta)
        print(faena + ': ' + url)
        # Streaming, so we can iterate over the response.
        try:
            response = requests.get(url,headers=headers, stream=True,cookies=r1.cookies, timeout=timeout)

            total_size_in_bytes= int(response.headers.get('content-length', 0))

            progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
            with open(archivos_temp + '/' + temp_name + '_' + faena + '.xls',"wb+") as file:
                for data in response.iter_content(block_size):
                    progress_bar.update(len(data))
                    file.write(data)
            progress_bar.close()
            if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
                print("ERROR, something went wrong")
        except requests.exceptions.Timeout:
            print('The request timed out')
            print('ERROR')

Andina: https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=11&desde=01/01/2020&hasta=01/01/2021&flota=-1&alerta=1&componente=M09


5.76MiB [00:22, 256kiB/s] 


Antucoya: https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=124&desde=01/01/2020&hasta=01/01/2021&flota=-1&alerta=1&componente=M09


100%|██████████| 2.20M/2.20M [00:12<00:00, 182kiB/s]


Caserones: https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=67&desde=01/01/2020&hasta=01/01/2021&flota=-1&alerta=1&componente=M09


...

11.2MiB [00:06, 1.62MiB/s]


Sierra Gorda: https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=65&desde=01/01/2020&hasta=01/01/2021&flota=-1&alerta=1&componente=M09


14.4MiB [00:47, 304kiB/s] 


SPENCE: https://scaaweb.komatsu.cl/generahistorico_faena.php?faena=125&desde=01/01/2020&hasta=01/01/2021&flota=-1&alerta=1&componente=M09


100%|██████████| 1.54M/1.54M [00:01<00:00, 1.05MiB/s]


In [85]:
import glob

temps = glob.glob(os.path.join(archivos_temp, "*.xls"))

df_consolidado = pd.read_csv(archivos_scaa + '/consolidado_scaa.csv', index_col='Nro.Muestra')
df_consolidado['Fecha Muestra'] = pd.to_datetime(df_consolidado['Fecha Muestra'])
df_consolidado['Fecha Cambio Aceite'] = pd.to_datetime(df_consolidado['Fecha Cambio Aceite'])
df_consolidado['Fecha Analisis'] = pd.to_datetime(df_consolidado['Fecha Analisis'])
# df_consolidado['Fecha Muestra'] = pd.to_datetime(df_consolidado['Fecha Muestra'],format='%d/%m/%Y')

In [86]:
for f in temps:
    xls_file = xlrd.open_workbook_xls(f, ignore_workbook_corruption=True, encoding_override="iso-8859-1")
    # read the csv file
    df_faena = pd.read_excel(xls_file, decimal=",", sheet_name=0, skiprows=3)
    # Columnas a utilizar
    if not df_faena.empty:
        df_faena = df_faena[['Nro.Muestra','Faena','Laboratorio','Modelo','Equipo','Componente','Posición','Fecha Muestra','Horas Aceite',
            'Fecha Cambio Aceite','Fecha Analisis','Horas Aceite Actual','Horometro Cambio Aceite','Intervalo Cambio de Aceite','Desviación en mantención', 
            'Nombre Aceite','Visc.40', 'Visc 100', 'Pto.Infl.', 'Diluc.', 'Agua', 'TBN', 'TAN', 'Hollin', 'Oxid.','Nitrac.', 'Sulf.', 
            'Ind.PQ.', 'AG', 'AL', 'B','CA', 'CR', 'CU', 'FE',  'K', 'MG', 'MO', 'NA',
            'NI', 'P', 'PB',  'SI', 'SN', 'TI', 'V', 'ZN','Detalle Alerta','Respuesta Alerta']]
        df_faena.set_index('Nro.Muestra', inplace=True)
        df_faena = df_faena[~df_faena.index.duplicated(keep='first')]
        df_faena['Fecha Muestra'] = pd.to_datetime(df_faena['Fecha Muestra'],format='%d/%m/%Y')
        df_faena['Fecha Cambio Aceite'] = pd.to_datetime(df_faena['Fecha Cambio Aceite'],format='%d/%m/%Y')
        df_faena['Fecha Analisis'] = pd.to_datetime(df_faena['Fecha Analisis'],format='%d/%m/%Y')
        df_faena['Alerta'] = (~df_faena['Detalle Alerta'].isna())*1
        df_faena.drop(['Detalle Alerta'], axis=1, inplace=True)
        df_faena['Modelo'] = df_faena.Modelo.str.replace("'","")
        df_faena.rename(columns = {'Modelo':'Flota'}, inplace = True)
        df_consolidado = df_consolidado.reindex(df_consolidado.index.union(df_faena.index))
        df_consolidado.update(df_faena)
    # df_consolidado = pd.concat([df_consolidado,df_faena])
    # df_consolidado = df_consolidado.combine_first(df_faena)
    # df_consolidado.update(df_faena)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

In [87]:
df_consolidado.to_csv(archivos_scaa + '/consolidado_scaa.csv')

In [88]:
# format_desde = dt.datetime.strptime(desde, '%d/%m/%Y').date()

updates['last_date_scaa'] = hoy.strftime('%d/%m/%Y')

with open(ruta_base +'//config.json', "w") as configFile:
    json.dump(config, configFile)

In [89]:
# df_consolidado['Fecha Muestra'] = pd.to_datetime(df_consolidado['Fecha Muestra'],format='%d/%m/%Y')
# filtro_fecha_scaa = (df_consolidado['Fecha Muestra'] > desde_actual) 
# df_consolidado_actual = df_consolidado[filtro_fecha_scaa]
# df_consolidado_actual.to_excel(archivos_scaa + '/consolidado_scaa_actual.xlsx')

In [90]:
df_consolidado[df_consolidado['Faena']=='Ministro Hales']

,Faena,Laboratorio,Flota,Equipo,Componente,Posición,Fecha Muestra,Horas Aceite,Fecha Cambio Aceite,Horas Aceite Actual,...,P,PB,SI,SN,TI,V,ZN,Respuesta Alerta,Alerta,Fecha Analisis
Nro.Muestra,,,,,,,,,,,,,,,,,,,,,
794064,Ministro Hales,DICTUC,930E-4,371,Motor Diesel,-----,2020-01-01,43805.0,2020-01-01,131.0,...,1185.00,1.7,8.00,0.10,0.0,0.1,1630.00,NaN,1.0,2020-01-04
794065,Ministro Hales,DICTUC,930E-4,372,Motor Diesel,-----,2020-01-02,1195.0,2019-10-13,251.0,...,1237.00,0.3,7.30,0.10,0.0,0.1,1553.00,NaN,0.0,2020-01-04
794066,Ministro Hales,DICTUC,930E-4,374,Motor Diesel,-----,2020-01-02,342.0,2019-12-08,475.0,...,1271.00,0.1,4.40,0.10,0.0,0.1,1554.00,NaN,1.0,2020-01-04
794067,Ministro Hales,DICTUC,930E-4,375,Motor Diesel,-----,2020-01-02,269.0,2019-12-17,55.0,...,1263.00,0.1,8.10,0.10,0.0,0.1,1554.00,NaN,0.0,2020-01-04
794068,Ministro Hales,DICTUC,930E-4,376,Motor Diesel,-----,2020-01-02,109.0,2019-12-22,0.0,...,1317.00,0.1,6.40,0.10,0.0,0.1,1651.00,NaN,0.0,2020-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028709,Ministro Hales,DICTUC,PC4000-6BHD,601.0,Mando Final,IZQ,2023-06-24,3444.0,2022-11-03,3444.0,...,305.07,0.0,21.05,0.00,0.0,0.0,23.56,NaN,1.0,2023-06-27
1028710,Ministro Hales,DICTUC,PC4000-6BHD,601.0,Sistema Hidraulico,-----,2023-06-24,14691.0,2020-08-15,14691.0,...,278.20,0.0,0.26,0.00,0.0,0.0,434.22,NaN,1.0,2023-06-27
1028711,Ministro Hales,DICTUC,PC4000-6BHD,601.0,Caja PTO Bomba,-----,2023-06-24,1469.0,2023-03-06,1469.0,...,272.64,0.0,20.21,0.00,0.0,0.0,4.83,NaN,0.0,2023-06-27


In [91]:
df_consolidado[(df_consolidado.Faena == 'Qblanca')].Flota.unique()

array(['D275AX-5EO', 'D375A-5', 'D475A-5E0', '830E-AC', 'GD825A-2',
       '730E', 'WD600-3', 'PC600LC-8', 'WA1200-6', 'WA1200-3', 'HD785-7',
       'D375A-6R', 'PC5500-6'], dtype=object)

In [92]:
df_consolidado[(df_consolidado.Faena == 'Qblanca') & (df_consolidado.Flota=='PC5500-6')].Equipo.unique()

array([906.0, '906.0', '904.0'], dtype=object)